In [32]:
import jdc
import math

import cv2
import os

import numpy as np
from numpy import linalg as LA

import torch
from torch import linalg as LAT

import scipy
from scipy import linalg as LAS
from scipy.stats import unitary_group
from scipy.stats import ortho_group

from tensorly import unfold
import imutils

In [47]:
class PM1NN:
    def __init__(self):
        self.name = "1NN Classifier"

    def __my_hosvd(self,A):
        dim = unfold(A,0)
        dim2 = unfold(A,1)
        dim3 = unfold(A,2)

        _,_,vh1 = LAS.svd(dim,full_matrices=False)
        _,_,vh2 = LAS.svd(dim2,full_matrices=False)
        _,_,vh3 = LAS.svd(dim3,full_matrices=False)

        return [np.matrix.getH(vh1),np.matrix.getH(vh2),np.matrix.getH(vh3)]

    def __stiefel_to_grassmann(self,stief):
        grass = []
        for i in range(len(stief)):
            orth = ortho_group.rvs(stief[i].shape[1])
            grass.append(np.dot(stief[i],orth))
        return grass

    def __get_geodisic_distance(self,theta):
        return LA.norm(np.sin(theta))

    def __get_canonical_angles(self,fm1,fm2):
        
        assert(fm1.shape[1] == fm2.shape[1])
        S = LAS.svdvals(fm1.T @ fm2)
        T= np.zeros(fm1.shape[1])
        for k in range(fm1.shape[1]):
            if S[k] < 10 ** -8:
                T[k] = np.sqrt(2*(1-S[k]))
            else:
                T[k] = np.real(np.arccos(S[k]))
        return np.flip(T)
    
    def __get_prod_geod2(self,gras_a,gras_b):
        
        assert(len(gras_a) == len(gras_b))
        canon = []
        for i in range(len(gras_a)):
            canon.append(self.__get_canonical_angles(gras_a[i],gras_b[i]))
        cart_prod = [(a,b,c) for a in canon[0] for b in canon[1] for c in canon[2]]
        sin = np.sin(np.array(cart_prod))
        return LA.norm(sin,ord='fro')

    def __get_prod_geod(self,gras_a,gras_b):
        
        if(len(gras_a) != len(gras_b)):
            print("Error")
            return
        geo_grass = np.zeros(len(gras_a))
        for i in range(len(gras_a)):
            canon = self.__get_canonical_angles(gras_a[i],gras_b[i])
            geo_grass[i] = self.__get_geodisic_distance(canon)
        return np.sum(geo_grass)
    
#         if(len(gras_a) != len(gras_b)):
#             print("Error")
#             return
#         geo_grass = np.zeros(len(gras_a))
#         prod_angles =  np.array([])
#         for i in range(len(gras_a)):
#             canon = self.__get_canonical_angles(gras_a[i],gras_b[i])
#             prod_angles = np.hstack((prod_angles,canon))
#         return LA.norm(np.sin(prod_angles))
  

    def fit(self,X_train,y_train):

        y = []
        self.X = []  
        for i in range(len(X_train)):
            for j in range(len(X_train[i])):
                y.append(j)
                prod_grassmann_x = self.__stiefel_to_grassmann(self.__my_hosvd(X_train[i][j]))
                
                self.X.append(prod_grassmann_x)
                
        self.y = y
        return self.X

    def predict(self,X_test):
        
        predictions = []
        
        for classX in range(len(X_test)):
            video = X_test[classX]
            test_grassmann = self.__stiefel_to_grassmann(self.__my_hosvd(video))
            geodesic_distances = []
            for x in range(len(self.X)):
                train_grassmann = self.X[x]
                geodesic_distances.append(self.__get_prod_geod(test_grassmann,train_grassmann))
            class_pred = self.y[np.argmin(geodesic_distances)]
            predictions.append(class_pred)


        return predictions